In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import ray
from ray import tune, train
from ray import air
from ray.tune import Trainable
from ray.tune.schedulers import PopulationBasedTraining
from ray.air.integrations.wandb import WandbLoggerCallback, setup_wandb
from ray.train import Checkpoint
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# Initialize Ray
ray.shutdown()  # Ensure Ray is reset
ray.init(ignore_reinit_error=True)
 
def parse_line(line):
    features, label = line.strip().split('||')
    features = [int(x) for x in features.split(',')]
    label = int(label)
    return {'features': features, 'label': label}
 
def load_data(file_path, num_lines=100000):
    parsed_data = []
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            if i >= num_lines:
                break
            parsed_data.append(parse_line(line))
    return ray.data.from_items(parsed_data)
 
 
class CustomNet(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes):
        super(CustomNet, self).__init__()
        self.layers = self._make_layers(input_size, hidden_layers, num_classes)
        self.to(device)
 
    def forward(self, x):
        return self.layers(x)
 
    def _make_layers(self, input_size, hidden_layers, num_classes):
        layers = [nn.Linear(input_size, hidden_layers[0]), nn.ReLU()]
        for i in range(len(hidden_layers) - 1):
            layers += [nn.Linear(hidden_layers[i], hidden_layers[i + 1]), nn.ReLU()]
        layers.append(nn.Linear(hidden_layers[-1], 1)) 
        return nn.Sequential(*layers)
 
    def reset_config(self, new_config):
        if 'hidden_layers' in new_config:
            self.update_architecture(new_config['hidden_layers'])
        return True
 
    def update_architecture(self, hidden_layers):
        # Extract the number of features for the input and output layers
        input_features = self.layers[0].in_features
        output_features = self.layers[-1].out_features
 
        # Create a new list of layers
        new_layers = [nn.Linear(input_features, hidden_layers[0]), nn.ReLU()]
 
        # Add the new hidden layers
        for i in range(1, len(hidden_layers)):
            new_layers.append(nn.Linear(hidden_layers[i - 1], hidden_layers[i]))
            new_layers.append(nn.ReLU())
 
        # Add the output layer
        new_layers.append(nn.Linear(hidden_layers[-1], output_features))
 
        # Update the model's layers
        self.layers = nn.Sequential(*new_layers)
 
# CHECKPOINT_DIR = r'C:\Users\lul\Desktop\MLBOT\uwumanoid_bot\checkpoints'
 
class CustomTrainable(Trainable):
    def setup(self, config):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.config = config
        self.model = CustomNet(config["input_size"], config["hidden_layers"], config["num_classes"]).to(self.device)
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=config["lr"])
        self._iteration = 0 
 
        # Load data here
        dataset = load_data(self.config['data_file_path'])
        train_dataset, test_dataset = dataset.train_test_split(test_size=0.2)
 
        # Convert to Torch Datasets
        self.train_torch_dataset = train_dataset.to_torch(
            label_column="label",
            feature_columns=["features"],
            batch_size=2048,
            unsqueeze_label_tensor=False
        )
        self.test_torch_dataset = test_dataset.to_torch(
            label_column="label",
            feature_columns=["features"],
            batch_size=2048,
            unsqueeze_label_tensor=False
        )
 
 
    def step(self):
        self.model.train()
        running_loss = 0.0
        num_batches = 0
 
        for features_tensor, labels_tensor in self.train_torch_dataset:
            # The tensors are already in the correct format
            features_tensor = features_tensor.to(self.device).float()
            labels_tensor = labels_tensor.to(self.device).float()
 
            self.optimizer.zero_grad()
            outputs = self.model(features_tensor).squeeze(1)
            outputs = outputs.squeeze(1)
            loss = self.criterion(outputs, labels_tensor)
            loss.backward()
            self.optimizer.step()
 
            running_loss += loss.item()
            num_batches += 1
            self._iteration += 1
 
            if num_batches % 10 == 0:
                print(f'Batch {num_batches}, Loss: {loss.item():.4f}')
 
 
        # Calculate average loss after all batches are processed
        epoch_loss = running_loss / num_batches if num_batches > 0 else 0
 
        # Evaluate the model on the test dataset for mean_accuracy
        mean_accuracy = evaluate_model(self.model, self.test_torch_dataset, self.device)
 
        # Checkpointing and Reporting
        if self._iteration % self.config["checkpoint_interval"] == 0:
            checkpoint_path = self.save_checkpoint(self.logdir)
            train.report({"mean_accuracy": mean_accuracy, "loss": epoch_loss, "checkpoint": checkpoint_path})
        else:
            train.report({"mean_accuracy": mean_accuracy, "loss": epoch_loss})
 
        print(f'Epoch {self._iteration}, Loss: {epoch_loss:.4f}, mean_accuracy: {mean_accuracy:.4f}')
 
        return {"mean_accuracy": mean_accuracy, "loss": epoch_loss}
 
 
            # Log metrics to WandB and Ray Tune
        # Note: Ensure WandbLoggerCallback is properly set up in your tune.run() call
        # wandb.log({"epoch": self._iteration, "loss": epoch_loss, "mean_accuracy": mean_accuracy})
 
    def save_checkpoint(self, checkpoint_dir):
        checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.pth")
        torch.save({
            "model_state": self.model.state_dict(),
            "optimizer_state": self.optimizer.state_dict()
        }, checkpoint_path)
        return checkpoint_path
 
 
    def load_checkpoint(self, checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        self.model.load_state_dict(checkpoint["model_state"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state"])
 
    def reset_config(self, new_config):
        # Update the model's architecture
        if 'hidden_layers' in new_config:
            self.model.update_architecture(new_config['hidden_layers'])
 
        return True
 
 
 
 
def mutate_layers(config, max_neurons_first_layer=256, min_neurons_last_layer=32):
    new_layers = config["hidden_layers"]
 
    mutation_prob = 0.5  # Probability of mutation
    layer_add_remove_prob = 0.5  # Probability of adding/removing a layer vs changing a layer size
 
    if np.random.rand() < mutation_prob:
        if np.random.rand() < layer_add_remove_prob:
            # Adding or removing a layer
            if len(new_layers) > 1 and np.random.rand() < 0.5:
                # Remove a layer with 50% probability if more than one layer exists
                new_layers.pop()
            else:
                # Add a new layer with a random size (limit the total number of layers if needed)
                if len(new_layers) < 5:  # Example limit for total number of layers
                    new_layer_size = np.random.choice([32, 64, 128, 256])
                    # Ensure the new layer size doesn't exceed the size of the last layer
                    if len(new_layers) > 0:
                        new_layer_size = min(new_layer_size, new_layers[-1])
                    new_layers.append(new_layer_size)
        else:
            # Changing the size of a random layer
            if new_layers:
                layer_to_change = np.random.randint(len(new_layers))
                new_layer_size = np.random.choice([32, 64, 128, 256])
                # Ensure the new layer size respects the size of adjacent layers
                if layer_to_change > 0:
                    new_layer_size = min(new_layer_size, new_layers[layer_to_change - 1])
                if layer_to_change < len(new_layers) - 1:
                    new_layer_size = max(new_layer_size, new_layers[layer_to_change + 1])
                new_layers[layer_to_change] = new_layer_size
 
    # Enforce constraints on the first and last layers
    new_layers[0] = min(new_layers[0], max_neurons_first_layer)
    if new_layers:
        new_layers[-1] = max(new_layers[-1], min_neurons_last_layer)
 
    return {"hidden_layers": new_layers}
 
 
def evaluate_model(model, test_torch_dataset, device):
    model.eval()
    correct = 0
    total = 0
 
    with torch.no_grad():
        for features, labels in test_torch_dataset:
            features = features.to(device).float()
            labels = labels.to(device).float()
 
            # Model predictions
            logits = model(features)
            probabilities = torch.sigmoid(logits)
            predictions = (probabilities > 0.5).float()
 
            # Update correct and total count
            correct += (predictions.squeeze() == labels).sum().item()
            total += labels.size(0)
 
             # # Debugging prints
            # print(f"Features shape: {features.shape}")
            # print(f"Labels shape: {labels.shape}")
            # print(f"Logits: {logits}")
            # print(f"Probabilities: {probabilities}")
            # print(f"Predictions: {predictions}")
            # print(f"Correct labels: {labels}")
            # # More debugging prints
            # print(f"Batch correct predictions: {(predictions.squeeze() == labels).sum().item()}")
            # print(f"Batch total: {labels.size(0)}")
            # print(f"Cumulative correct predictions: {correct}")
            # print(f"Cumulative total: {total}")
 
 
 
    mean_accuracy = correct / total if total > 0 else 0
    print(f"Final mean accuracy: {mean_accuracy}")
    return mean_accuracy
 
 
 
 
 
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="mean_accuracy",
    mode="max",
    perturbation_interval=5,
    hyperparam_mutations={
        #"lr": tune.loguniform(1e-4, 1e-1),
        "hidden_layers": mutate_layers  
    }
)
 
analysis = tune.run(
    CustomTrainable,
    name="pbt_test",
    scheduler=scheduler,
    num_samples=1,
    reuse_actors=True,
    config={
        "lr": 0.001,
        "checkpoint_interval": 5,
        "num_epochs": 10,
        "input_size": 173,
        "num_classes": 1,
        "hidden_layers": [64],
        "data_file_path": r'C:\Users\lul\Desktop\MLBOT\games\ML1_50m_games.txt',
        # Include other configurations here
    },
   stop={"training_iteration": 1000, "mean_accuracy": 0.8},
 
    resources_per_trial={
        "cpu": 1,
        "gpu": 0.25
    },
    callbacks=[WandbLoggerCallback(project="is-project", group="is-project", api_key="3a2d5a882da8bbcd3651c1ef4a6009a863d89d16")]
)
 
 
top_trial = analysis.get_best_trial(metric="mean_accuracy", mode="max", scope="last-5-avg")
model = CustomNet(top_trial.config["input_size"], top_trial.config["hidden_layers"], top_trial.config["num_classes"]).to(device)
checkpoint = analysis.get_best_checkpoint(top_trial, metric="mean_accuracy", mode="max")
if checkpoint and os.path.isfile(checkpoint):
    model.load_state_dict(torch.load(checkpoint))
else:
    print(f"Checkpoint file not found at {checkpoint}")
 
model_save_directory = r'C:\Users\lul\Desktop\MLBOT\uwumanoid_bot\models'
layer_info = "_".join(map(str, top_trial.config["hidden_layers"]))
model_file_name = f'model_{layer_info}.pth'
model_save_path = os.path.join(model_save_directory, model_file_name)
 
torch.save(model.state_dict(), model_save_path)
print(f"Best model saved to {model_save_path}")

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import ray
from torch.utils.data import TensorDataset, DataLoader, random_split
from ray import train, tune
from ray.tune.schedulers import PopulationBasedTraining
from ray.air.integrations.wandb import WandbLoggerCallback, setup_wandb

file_path = 'path_to_your_file/random_random_10k_games.txt' 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Ray
ray.shutdown()  # Ensure Ray is reset
ray.init(ignore_reinit_error=True)

# Load and parse the data
total_lines = sum(1 for line in open(file_path, 'r'))
data_list = []
labels_list = []
with open(file_path, 'r') as file:
    for i, line in enumerate(file, 1):
        features, label = line.strip().split('||')
        features = [int(x) for x in features.split(',')]
        label = int(label)
        data_list.append(features)
        labels_list.append(label)
        
        if i % 100000 == 0:
            percentage_done = (i / total_lines) * 100
            print(f"Processed {i} lines ({percentage_done:.2f}% completed)")

print(f"Processed {total_lines} lines (100% completed)")

# Convert lists to NumPy arrays and then to PyTorch tensors
data_np = np.array(data_list, dtype=np.float32)
labels_np = np.array(labels_list, dtype=np.int64)  
data_tensor = torch.from_numpy(data_np)
labels_tensor = torch.from_numpy(labels_np)
print("Data shape:", data_tensor.shape)

# Create a TensorDataset and split it
dataset = TensorDataset(data_tensor, labels_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for the training and testing sets
batch_size = 2048
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


class CustomNet(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes):
        super(CustomNet, self).__init__()
        self.layers = self._make_layers(input_size, hidden_layers, num_classes)
        self.to(device)

    def forward(self, x):
        return self.layers(x)

    def _make_layers(self, input_size, hidden_layers, num_classes):
        layers = [nn.Linear(input_size, hidden_layers[0]), nn.ReLU()]
        for i in range(len(hidden_layers) - 1):
            layers += [nn.Linear(hidden_layers[i], hidden_layers[i + 1]), nn.ReLU()]
        layers.append(nn.Linear(hidden_layers[-1], num_classes))
        return nn.Sequential(*layers)

    def reset_config(self, new_config):
        if 'hidden_layers' in new_config:
            self.update_architecture(new_config['hidden_layers'])
        return True
    
    def update_architecture(self, hidden_layers):
        # Extract the number of features for the input and output layers
        input_features = self.layers[0].in_features
        output_features = self.layers[-1].out_features

        # Create a new list of layers
        new_layers = [nn.Linear(input_features, hidden_layers[0]), nn.ReLU()]

        # Add the new hidden layers
        for i in range(1, len(hidden_layers)):
            new_layers.append(nn.Linear(hidden_layers[i - 1], hidden_layers[i]))
            new_layers.append(nn.ReLU())

        # Add the output layer
        new_layers.append(nn.Linear(hidden_layers[-1], output_features))

        # Update the model's layers
        self.layers = nn.Sequential(*new_layers)


def train_model(config, checkpoint_dir=None):
    wandb = setup_wandb(config, project="is-project")
    model = CustomNet(config["input_size"], config["hidden_layers"], config["num_classes"]).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    if checkpoint_dir:
        model_checkpoint = os.path.join(checkpoint_dir, "model.pth")
        if os.path.isfile(model_checkpoint):
            model.load_state_dict(torch.load(model_checkpoint))

    for epoch in range(config["num_epochs"]):
        model.train()
        running_loss = 0.0
        for batch_idx, (inputs, labels) in enumerate(train_loader, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if batch_idx % 10 == 0:
                print(f'Epoch [{epoch+1}/{config["num_epochs"]}], '
                      f'Batch [{batch_idx}/{len(train_loader)}], '
                      f'Loss: {loss.item():.4f}')

        # Check and apply new configuration at the end of each epoch
        new_config = ray.train.get_context().get_trial_dir()  # Get new configuration from PBT
        if new_config:
            model.reset_config(new_config)  # Apply new configuration to the model

        epoch_loss = running_loss / len(train_loader)
        accuracy = evaluate_model(model, test_loader)
        
        wandb.log({"epoch": epoch, "loss": epoch_loss, "accuracy": accuracy})
        train.report({"accuracy": accuracy, "loss": epoch_loss})
        
        print(f'Epoch [{epoch+1}/{config["num_epochs"]}], '
              f'Loss: {epoch_loss:.4f}, '
              f'Accuracy: {accuracy:.4f}')

        # Save checkpoint at the end of each epoch
        with train.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "model.pth")
            torch.save(model.state_dict(), path)

    return model


def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def mutate_layers(config, max_neurons_first_layer=256, min_neurons_last_layer=32):
    new_layers = config["hidden_layers"]
    
    mutation_prob = 0.5  # Probability of mutation
    layer_add_remove_prob = 0.5  # Probability of adding/removing a layer vs changing a layer size

    if np.random.rand() < mutation_prob:
        if np.random.rand() < layer_add_remove_prob:
            # Adding or removing a layer
            if len(new_layers) > 1 and np.random.rand() < 0.5:
                # Remove a layer with 50% probability if more than one layer exists
                new_layers.pop()
            else:
                # Add a new layer with a random size (limit the total number of layers if needed)
                if len(new_layers) < 5:  # Example limit for total number of layers
                    new_layer_size = np.random.choice([32, 64, 128, 256])
                    # Ensure the new layer size doesn't exceed the size of the last layer
                    if len(new_layers) > 0:
                        new_layer_size = min(new_layer_size, new_layers[-1])
                    new_layers.append(new_layer_size)
        else:
            # Changing the size of a random layer
            if new_layers:
                layer_to_change = np.random.randint(len(new_layers))
                new_layer_size = np.random.choice([32, 64, 128, 256])
                # Ensure the new layer size respects the size of adjacent layers
                if layer_to_change > 0:
                    new_layer_size = min(new_layer_size, new_layers[layer_to_change - 1])
                if layer_to_change < len(new_layers) - 1:
                    new_layer_size = max(new_layer_size, new_layers[layer_to_change + 1])
                new_layers[layer_to_change] = new_layer_size

    # Enforce constraints on the first and last layers
    new_layers[0] = min(new_layers[0], max_neurons_first_layer)
    if new_layers:
        new_layers[-1] = max(new_layers[-1], min_neurons_last_layer)

    return {"hidden_layers": new_layers}

scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="accuracy",
    mode="max",
    perturbation_interval=5,
    hyperparam_mutations={
        #"lr": tune.loguniform(1e-4, 1e-1),
        "hidden_layers": mutate_layers  
    }
)

analysis = tune.run(
    lambda config: train_model(config),
    name="pbt_test",
    scheduler=scheduler,
    num_samples=4,
    config={
        "lr": 0.001,  # Fixed learning rate
        "num_epochs": 10,
        "input_size": 173,  # Updated to match the number of features
        "num_classes": 2,  # This remains 2 for binary classification (won or lost)
        "hidden_layers": [64],  # Initial architecture
    },
    resources_per_trial={
            "cpu": 1, # 1 CPU core
            "gpu": 0.25  # 1/4 GPU
        },
    callbacks=[WandbLoggerCallback(project="is-project", group="is-project", api_key="3a2d5a882da8bbcd3651c1ef4a6009a863d89d16")]
)

top_trials = analysis.get_best_trials(metric="accuracy", mode="max", limit=5)
for idx, trial in enumerate(top_trials, start=1):
    model = CustomNet(trial.config["input_size"], trial.config["hidden_layers"], trial.config["num_classes"]).to(device)
    checkpoint = analysis.get_best_checkpoint(trial)
    model.load_state_dict(torch.load(checkpoint))

    # Create a dynamic model name based on architecture
    layer_info = "_".join(map(str, trial.config["hidden_layers"]))
    model_save_path = f'model_{layer_info}_{idx}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

KeyboardInterrupt: 